## Langchain

##installation

In [ ]:

!pip install groq
!pip install langchain-groq
!pip install langchain-huggingface
!pip install langchain-text-splitters
!pip install langchain-community
!pip install chromadb

##Integration des llms

In [ ]:

import os
from getpass import getpass

# Prompt to enter the Groq API key securely (won't show on screen)
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")


Enter your Groq API key: ··········


In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="openai/gpt-oss-20b")
response = llm.invoke("salut")
print(response.content)


Salut ! Comment ça va aujourd’hui ?


### Prompt

c'est le prompt alors pour avoir des prompt plus avances langcahin offre des  promptemplates qui permettent de personaliser les prompt.Donc les promptemplates ces sont des composant pour generer des prompt  en introduire de variables , des places holders extra




In [ ]:
from langchain import PromptTemplate

prompt = PromptTemplate.from_template("Traduire la phrase suivante\"{phrase}\" en {langue}")
prompt_formatted = prompt.format(phrase = "Bonjour comment ca va", langue ="Anglais")
print(prompt_formatted)

Traduire la phrase suivante"Bonjour comment ca va" en Anglais


In [ ]:
print(llm.invoke(prompt_formatted).content)

**English translation:**  
"Hello, how are you?"


Les autres  qui sont interrissantes :

ChatPromptTemplate	Template qui génère une liste de messages structurés avec rôles (system, human, assistant, etc).	Pour les modèles chat, avec instructions système et user.

MessagesPlaceholder	Permet d’insérer dynamiquement une liste de messages (ex: historique de conversation).	Gérer l’historique ou insérer des messages variables.

### Connecter les donnes


Chargement

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader('/content/histoirep.txt')
docs = loader.load()


Transfromation:Chunking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
spliter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)
chunks = spliter.split_documents(docs)
print("len chunks", len(chunks))


len chunks 4



Embedding



In [ ]:
# Prompt to enter the Groq API key securely (won't show on screen)
import os
from getpass import getpass
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter your HF API key: ")

Enter your HF API key: ··········


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

BdVectorielle

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(chunks, embedding_model, collection_name="histoire_collection")
print("Données enregistrées dans la base vectorielle Chroma.")

Données enregistrées dans la base vectorielle Chroma.


In [ ]:
query = "qui est Sophie?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

À Paris, dans le quartier animé du Marais, le Café des Lilas est un petit bistrot où les habitués se retrouvent. Chaque matin, Sophie, une libraire passionnée de 32 ans, s'installe à une table près de


##chains

In [ ]:
from langchain import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(phrase="Bonjour comment ca va", langue="Anglais")

'Hello, how are you?'

- **SimpleSequentialChain** : relie plusieurs chaînes de façon linéaire (ex. un LLM qui rédige → un LLM qui traduit).
- **SequentialChain** (avancé) : permet de composer plusieurs étapes en gérant les variables de sortie/entrée entre elles.